Inizializzazione Rete Neurale e prima run in assenza di algoritmi evolutivi, usando parametri casuali

In [ ]:
import random
import numpy as np

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Fa4H/computational_intelligence/refs/heads/main/brain_tumor.csv")
df.head()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import datasets
from sklearn.neural_network import MLPClassifier

X, y = df.iloc[:, 2:].copy(), df.iloc[:, 1].copy()

X_tr_val, X_test, y_tr_val, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_tr_val, y_tr_val, test_size = 0.2, random_state = 1, stratify = y_tr_val)

vals, counts = np.unique(y_val, return_counts=True)
#print(vals)    # → valori unici
#print(counts)
nums_0 = counts[0]
nums_1 = counts[1]

In [ ]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_val_std = sc.transform(X_val)
X_test_std = sc.transform(X_test)
print(X_train_std[:3])

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10,10), max_iter=100, activation="logistic", alpha=0.1, solver="sgd", learning_rate="constant", learning_rate_init=0.001, random_state=1, shuffle=True, tol=0.0001, verbose=True, warm_start=False)

mlp.fit(X_train_std, y_train)

In [ ]:
pred_val = mlp.predict(X_val_std)
c = confusion_matrix(pred_val, y_val)
print(classification_report(pred_val, y_val, zero_division=0))
print(c)

Definizione algoritmo evolutivo genetico e implementazione nella rete neurale

In [ ]:
def hidden_layers_encoding():
  numlays = np.random.randint(1, 3)
  a = [np.random.randint(1, 100) for i in range(numlays)]
  return a

def max_iter_encoding():
  return np.random.randint(1, 500)

def activation_encoding():
  return np.random.randint(0, 3)

def activation_decoding(gene_value):
    if not hasattr(activation_decoding, "_phenotypes"):
        activation_decoding._phenotypes = ("relu", "tanh", "logistic", "identity")  # solo alla prima chiamata
    return activation_decoding._phenotypes[gene_value] # riusa il dizionario nelle chiamate successive

def alpha_encoding(min_alpha = 10**-4, max_alpha = 10**0, exp_step = 0.01):
    """
    encoding: alpha appartenente a range di Reali (phenotype space) --->  numero di step appartenente a sottoinsieme di Interi (genotype space)
    -------------------------------------------------------------------------------------------------------------------------------------------
    La funzione di encoding genera un numero naturale tra 0 ed un numero intero massimo che è determinato
    dal numero totale ammissibile di valori distinti discretizzati di alpha, tenendo in conto
    la restrizione data dal range [min_alpha, max_alpha] e exp_step.
    Considerando la discretizzazione lineare nella potenza di 10, questo intero corrisponde semanticamente al numero di exp_step da aggiungere
    a min_exp per ottenere l'esponente da dare a 10 per ottenere alpha.

    """
    # Calcolo di esponente minimo, massimo e numero di step in questo range
    min_exp = np.log10(min_alpha)
    max_exp = np.log10(max_alpha)
    num_step = int((max_exp - min_exp) / exp_step)
    # Estraggo il valore del gene: un valore intero randomico di step da fare
    gene_value = random.randint(0, num_step)

    return gene_value

def alpha_decoding(gene_value, min_alpha = 10**-4, max_alpha = 10**0, exp_step = 0.01):
    """
    decoding: numero di step appartenente a sottoinsieme di Interi (genotype space) ---> alpha appartenente a range di Reali (phenotype space)
    -------------------------------------------------------------------------------------------------------------------------------------------
    La funzione di decoding assume il valore intero del gene (gene_value) e lo associa semanticamente al numero
    di exp_step che bisogna aggiungere al min_exp per ottenere l'esponente da dare a 10 per ottenere il corrispondente alpha.
    alpha = 10**(min_exp + gene_value*exp_step) -- lineare nell'esponente --

    """
    # Calcolo di esponente minimo, massimo e numero di step in questo range
    min_exp = np.log10(min_alpha)
    max_exp = np.log10(max_alpha)
    num_step = int((max_exp - min_exp) / exp_step)
    # Calcolo l'esponente da dare a 10 partendo dal valore minimo e aggiungendo ad esso un numero "gene_value" di exp_step (è lineare nell'esponente)
    esp = min_exp + gene_value*exp_step
    # Calcolo alpha
    alpha = 10**esp

    return alpha

def learning_rate_init_encoding(min_lri = 10**-4,  max_lri = 10**-2,  exp_step = 0.01):
    """
    encoding: lri appartenente a range di Reali (phenotype space) --->  numero di step appartenente a sottoinsieme di Interi (genotype space)
    -------------------------------------------------------------------------------------------------------------------------------------------
    La funzione di encoding genera un numero naturale tra 0 ed un numero intero massimo che è determinato
    dal numero totale ammissibile di valori distinti discretizzati di learning_rate_init, tenendo in conto
    la restrizione data dal range [min_lri, max_lri] e exp_step.
    Considerando la discretizzazione lineare nella potenza di 10, questo intero corrisponde semanticamente al numero di exp_step da aggiungere
    a min_exp per ottenere l'esponente da dare a 10 per ottenere learning_rate_init.

    """
    # Calcolo di esponente minimo, massimo e numero di step in questo range
    min_exp = np.log10(min_lri)
    max_exp = np.log10(max_lri)
    num_step = int((max_exp - min_exp) / exp_step)
    # Estraggo il valore del gene: un valore intero randomico di step da fare
    gene_value = random.randint(0, num_step)

    return gene_value

def learning_rate_init_decoding(gene_value, min_lri = 10**-4,  max_lri = 10**-2,  exp_step = 0.01):
    """
    decoding: numero di step appartenente a sottoinsieme di Interi (genotype space) ---> lri appartenente a range di Reali (phenotype space)
    -------------------------------------------------------------------------------------------------------------------------------------------
    La funzione di decoding assume il valore intero del gene (gene_value) e lo associa semanticamente al numero
    di exp_step che bisogna aggiungere al min_exp per ottenere l'esponente da dare a 10 per ottenere il corrispondente learning_rate_init.
    lri = 10**(min_exp + gene_value*exp_step) -- lineare nell'esponente --

    """
    # Calcolo di esponente minimo, massimo e numero di step in questo range
    min_exp = np.log10(min_lri)
    max_exp = np.log10(max_lri)
    num_step = int((max_exp - min_exp) / exp_step)
    # Calcolo l'esponente da dare a 10 partendo dal valore minimo e aggiungendo ad esso un numero "gene_value" di exp_step (è lineare nell'esponente)
    esp = min_exp + gene_value*exp_step
    # Calcolo lri
    lri = 10**esp

    return lri

def tol_encoding(min_tol = 10**-8,  max_tol = 10**-4,  exp_step = 0.01):
    """
    encoding: tol appartenente a range di Reali (phenotype space) --->  numero di step appartenente a sottoinsieme di Interi (genotype space)
    -------------------------------------------------------------------------------------------------------------------------------------------
    La funzione di encoding genera un numero naturale tra 0 ed un numero intero massimo che è determinato
    dal numero totale ammissibile di valori distinti discretizzati di tol, tenendo in conto
    la restrizione data dal range [min_tol, max_tol] e exp_step.
    Considerando la discretizzazione lineare nella potenza di 10, questo intero corrisponde semanticamente al numero di exp_step da aggiungere
    a min_exp per ottenere l'esponente da dare a 10 per ottenere tol.

    """
    # Calcolo di esponente minimo, massimo e numero di step in questo range
    min_exp = np.log10(min_tol)
    max_exp = np.log10(max_tol)
    num_step = int((max_exp - min_exp) / exp_step)
    # Estraggo il valore del gene: un valore intero randomico di step da fare
    gene_value = random.randint(0, num_step)

    return gene_value

def tol_decoding(gene_value, min_tol = 10**-8,  max_tol = 10**-4,  exp_step = 0.01):
    """
    decoding: numero di step appartenente a sottoinsieme di Interi (genotype space) ---> tol appartenente a range di Reali (phenotype space)
    -------------------------------------------------------------------------------------------------------------------------------------------
    La funzione di decoding assume il valore intero del gene (gene_value) e lo associa semanticamente al numero
    di exp_step che bisogna aggiungere al min_exp per ottenere l'esponente da dare a 10 per ottenere il corrispondente tol.
    tol = 10**(min_exp + gene_value*exp_step) -- lineare nell'esponente --

    """
    # Calcolo di esponente minimo, massimo e numero di step in questo range
    min_exp = np.log10(min_tol)
    max_exp = np.log10(max_tol)
    num_step = int((max_exp - min_exp) / exp_step)
    # Calcolo l'esponente da dare a 10 partendo dal valore minimo e aggiungendo ad esso un numero "gene_value" di exp_step (è lineare nell'esponente)
    esp = min_exp + gene_value*exp_step
    # Calcolo lri
    tol = 10**esp

    return tol

encoding = [hidden_layers_encoding, max_iter_encoding, activation_encoding, alpha_encoding, learning_rate_init_encoding, tol_encoding]

In [ ]:
def age_based(offspring, pop):
    return offspring
def fitness_based(offspring, pop):
    return tools.selBest(offspring + pop, k=len(pop))

def new_pop_decoding(gene_value):
    if not hasattr(new_pop_decoding, "_phenotypes"):
        new_pop_decoding._phenotypes = (age_based, fitness_based)  # solo alla prima chiamata
    return new_pop_decoding._phenotypes[gene_value] # riusa il dizionario nelle chiamate successive

In [ ]:
def setup_problem():
    creator.create("FitnessMax", base.Fitness, weights = (1.0,))
    creator.create("Individual", list, fitness = creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initCycle, creator.Individual, encoding, n = 1)

    def evalFitness(individual):
        # decoding (inverse representation to create the corresponding phenotype)
        hidden_layers = individual[0]
        max_iter = individual[1]
        activation = activation_decoding(individual[2])
        alpha = alpha_decoding(individual[3])
        learning_rate_init = learning_rate_init_decoding(individual[4])
        tol = tol_decoding(individual[5])
        # costruzione e apprendimento della NN
        mlp = MLPClassifier(hidden_layer_sizes=hidden_layers, activation=activation, alpha=alpha, max_iter=max_iter, learning_rate_init=learning_rate_init, tol=tol)
        mlp.fit(X_train_std, y_train)
        # validazione
        pred_val = mlp.predict(X_val_std)
        # quality measure in the phenotype space
        report = classification_report(pred_val, y_val, zero_division=0,output_dict=True)
        f1_class0 = report['0']['f1-score']
        f1_class1 = report['1']['f1-score']
        f1_weighted = (f1_class0*nums_0 + f1_class1*nums_1)/(nums_0 + nums_1)
        return f1_weighted,

    toolbox.register("evaluate", evalFitness)

    return toolbox

toolbox_problema = setup_problem()


In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Filtra il ConvergenceWarning di scikit-learn
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
def GA(toolbox = toolbox_problema , popSize = 10, CXPB = 0.7, MUTPB = 0.3, indpbMUT = 0.1 , maxEvals = 100, maxGens = None, cxFUNC = tools.cxTwoPoint, mutFUNC = mutCustom1, mut_params=None, selFUNC = tools.selTournament, new_pop = age_based, tournsize = 3, verbose = False):

    # Registriamo altri oggetti caratteristici dell'algoritmo e non del problema
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", cxFUNC)
    toolbox.register("mutate", mutFUNC, indpb = indpbMUT)
    toolbox.register("parent_select", selFUNC, tournsize = tournsize)
    toolbox.register("next_gen", new_pop)


    # definizione statistiche
    stats = tools.Statistics(key = lambda ind: ind.fitness.values)
    def Mean(L):
        return round(np.mean(L), 2)
    def Std(L):
        return round(np.std(L), 2)
    def Min(L):
        return round(np.min(L), 2)
    def Max(L):
        return round(np.max(L), 2)

    stats.register("avg", Mean)
    stats.register("std", Std)
    stats.register("min", Min)
    stats.register("max", Max)

    halloffame = tools.HallOfFame(1)

    logbook = tools.Logbook()
    logbook.header = ["gen", "nevals"] + stats.fields

    # creazione della popolazione
    pop = toolbox.population(popSize)

    invalid_ind = [ind for ind in pop if not ind.fitness.valid]
    fitness = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitness):
        ind.fitness.values = fit


    halloffame.update(pop)
    rec = stats.compile(pop)
    logbook.record(gen = 0, nevals = len(invalid_ind) , **rec)
    if verbose:
        print(logbook.stream)
    g = 0
    while np.sum(logbook.select("nevals")) < maxEvals:
        g += 1
        # Select the next generation of parents. These are only links to the individuals of pop. Then, need to deep clone them to avoid changes in place
        sel_offspring = toolbox.parent_select(pop, len(pop)) # selects n(=len(pop)) individuals from n with duplicates allowed
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, sel_offspring))
        # Apply crossover on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]): # Parents early selected will be the childrens because toolbox.mate changes them in place
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        # Apply mutation on the offspring
        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the halloffame with the generated individuals
        halloffame.update(offspring)
        # The population is entirely replaced by the offspring (age-based selection) or chosen from best fitness individuals
        pop[:] = toolbox.next_gen(offspring, pop)
        # Append the current generation statistics to the logbook
        rec = stats.compile(pop)
        logbook.record(gen = g, nevals = len(invalid_ind), **rec)
        if verbose:
            print(logbook.stream)
    return pop, logbook, halloffame[0],  halloffame[0].fitness.values[0]

pops, logb, hof, hoffit = GA()


In [ ]:
hof, hoffit

il GA ha i seguenti geni:
- popSize: range [5, 20]
- CXPB: [0.1, 0.9]
- MUTPB: [0.05, 0.5]
- indpbMUT: [0.05, 0.5]
- maxEvals: [popSize, 10*popSize]
- cxFUNC: [cxOnePoint, cxTwoPoint, cxUniform] se si usa cxUniform serve l'attributo indpbCX
- indpbCX: [0.2, 0.8] 
- mutFUNC: [mutReinit, mutGaussianRound, mutDecay]
- selFunc: [selTournament, selRoulette, selRandom, selBest] se si usa selTournament serve l'attributo tournsize
- tournsize: tourn = [int(0.3*popSize), int(0.5*popSize)]   con restrizione toursize = max(3, tourn)   
- new_pop: [age_based, fitness_based]

In [ ]:
# FUNZIONI DI ENCODING E DECODING

def popSize_encoding():
    popsize = random.randint(5, 20)
    return popsize

def CXPB_encoding(min_CXPB = 0.1, max_CXPB = 0.9, step = 0.1):
    num_step = int((max_CXPB - min_CXPB) / step)
    gene_value = random.randint(0, num_step)
    return gene_value

def CXPB_decoding(gene_value, min_CXPB = 0.1, max_CXPB = 0.9, step = 0.1):
    cxpb = min_CXPB + gene_value*step
    return cxpb

def MUTPB_encoding(min_MUTPB = 0.05, max_MUTPB = 0.5, step = 0.01):
    num_step = int((max_MUTPB - min_MUTPB) / step)
    gene_value = random.randint(0, num_step)
    return gene_value

def MUTPB_decoding(gene_value, min_MUTPB = 0.05, max_MUTPB = 0.5, step = 0.01):
    mutpb = min_MUTPB + gene_value*step
    return mutpb

def indpbMUT_encoding(min_indpbMUT = 0.05, max_indpbMUT = 0.5, step = 0.05):
    num_step = int((max_indpbMUT - min_indpbMUT) / step)
    gene_value = random.randint(0, num_step)
    return gene_value

def indpbMUT_decoding(gene_value, min_indpbMUT = 0.05, max_indpbMUT = 0.5, step = 0.05):
    indpbMUT = min_indpbMUT + gene_value*step
    return indpbMUT
popSize = popSize_encoding()

def maxEvals_encoding(min_maxEvals = popSize, max_maxEvals = 10*popSize): # DA DOVE PRENDO POPSIZE NELLA CREAZIONE DEL GENE?
    maxEvals = random.randint(min_maxEvals, max_maxEvals)
    return maxEvals

def eval():
    popsize = decoding(ind[0])
    mutpb()

In [ ]:
def eval():
    cxFunc = individual[3]
    indpbcx = None
    if cxFunc == "cxUniform":
        indpbcx = 0.5
    

    GA(cxfunc = cxFunc, indpbcx = indpcx)
def eval():
    cxFunc = individual[3]
    indpbcx = individual[4] 
    if cxFunc != "cxUniform":
        indpbcx = None

encoding = [hidden_layers_encoding, max_iter_encoding, activation_encoding, alpha_encoding, learning_rate_init_encoding, tol_encoding]

def mutCustom1(individual, indpb):
    for i in encoding:
        if random.random() < indpb:
            individual[i] = encoding[i]()
    return individual

def mutGaussianRound(individual, indpb):
    if random.random() < indpb:
        # Applica mutazione gaussiana al primo gene (numero di neuroni per strato)
        for i in range(len(individual[0])):
            mutated_value = individual[0][i] + random.gauss(0, (bound[1]-bound[0])*10/100)
            # Arrotonda al numero intero più vicino
            mutated_value = int(round(mutated_value))
            # Clampa il valore entro i limiti validi
            mutated_value = max(bound[0], min(mutated_value, bound[1]))
            individual[0][i] = mutated_value
    for i in range(1, len(individual)):
        if random.random() < indpb:
            # Applica rumore gaussiano
            mutated_value = individual[i] + random.gauss(0, (bound[i+1]-bound[i])*10/100)
            # Arrotonda al numero intero più vicino
            mutated_value = int(round(mutated_value))
            # Clampa il valore entro i limiti validi per quel gene
            mutated_value = max(bound[i], min(mutated_value, bound[i+1]))
            individual[i] = mutated_value
    return individual

def mutRandomDecay(individual, current_evals, maxEvals, indpb):
    """
        Mutazione che decresce nel tempo.
    """
    # Calcola il fattore di decrescita basato sul numero di valutazioni correnti
    decay_factor = 1 - current_evals / maxEvals

    if random.random() < indpb:
        # Applica mutazione gaussiana al primo gene (numero di neuroni per strato)
        for i in range(len(individual[0])):
            # Applica la mutazione decrescente
            mut = random.random(-1, 1)*individual[0][i]*decay_factor + individual[0][i]
            # Clampa il valore entro i limiti validi per quel gene
            individual[0][i] = max(bound(0), min(mut, bound(1)))
    for i in range(1, len(individual)):
        if random.random() < indpb:
            # Applica la mutazione decrescente
            mut = random.random(-1, 1)*individual[i]*decay_factor + individual[i]
            # Clampa il valore entro i limiti validi per quel gene
            individual[i] = max(bound(i), min(mut, bound(i+1)))
    return individual

In [ ]:
cxFuncde = decoding(individual[3]) 
indpbcx = individual[4]
toursizede = individual[5]
GA(cxFunc = cxFuncde, tournsize = toursizede)
    if cxFunc == tools.cxUniform:
        toolbox.register("mate", cxFunc, indpb=indpbcx )
    else:
        toolbox.register("mate", cxFunc)

    if selFunc == tools.selTournament:
        toolbox.register("parent_select", tools.selTournament, tournsize = tournsize, k = popSize)
    else:
        toolbox.register("parent_select", selFunc, k = popSize)
        